In [1]:
import datajoint as dj
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from ibl_pipeline.plotting import plotting_utils_ephys as putils
import scipy.signal as signal
from ibl_pipeline.plotting import ephys as ephys_plotting

Connecting shan@datajoint.internationalbrainlab.org:3306
ONE not set up


In [2]:
behavior = dj.create_virtual_module('behavior', 'ibl_behavior')
ephys = dj.create_virtual_module('ephys', 'ibl_ephys')

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:212: UserWarning:

MySQL server has gone away. Reconnecting to the server.



In [5]:
import datetime
from uuid import UUID
key = {'subject_uuid': UUID('15f98ab5-20d8-4345-a5de-21912a39575f'),
 'session_start_time': datetime.datetime(2020, 1, 20, 14, 56, 51),
 'probe_idx': 0,
 'cluster_revision': '0',
 'cluster_id': 30}

In [6]:
trials_all = (behavior.TrialSet.Trial * ephys.TrialSpikes & key).proj(
            'trial_start_time', 'trial_stim_on_time',
            'trial_response_time', 'trial_feedback_time',
            'trial_response_choice', 'trial_spike_times',
            trial_duration='trial_end_time-trial_start_time',
            trial_signed_contrast='trial_stim_contrast_right - trial_stim_contrast_left'
        ) & 'trial_duration < 5' & 'trial_response_choice!="No Go"'

/usr/local/lib/python3.6/dist-packages/datajoint/connection.py:212: UserWarning:

MySQL server has gone away. Reconnecting to the server.



In [7]:
trials = trials_all
trial_type = 'all'
align_event = 'stim on'
bin_size = 0.025
smoothing = 0.025
x_lim = [-1, 1]

In [8]:
if trial_type == 'left':
    color = 'green'
elif trial_type == 'right':
    color = 'blue'
elif trial_type == 'all':
    color = 'black'
elif trial_type == 'incorrect':
    color = 'red'
else:
    raise NameError('Invalid type name')

In [9]:
n_offset = 5 * int(np.ceil(smoothing / bin_size))  # get rid of boundary effects for smoothing
n_bins_pre = int(np.ceil(np.negative(x_lim[0]) / bin_size)) + n_offset
n_bins_post = int(np.ceil(x_lim[1] / bin_size)) + n_offset
n_bins = n_bins_pre + n_bins_post

In [10]:
spk_times = trials.fetch('trial_spike_times')

hist = np.histogram(np.hstack(spk_times),
                    range=x_lim,
                    bins=n_bins)

/usr/local/lib/python3.6/dist-packages/numpy/lib/histograms.py:824: RuntimeWarning:

invalid value encountered in greater_equal

/usr/local/lib/python3.6/dist-packages/numpy/lib/histograms.py:825: RuntimeWarning:

invalid value encountered in less_equal



In [11]:
hist[1]

array([-1.        , -0.97777778, -0.95555556, -0.93333333, -0.91111111,
       -0.88888889, -0.86666667, -0.84444444, -0.82222222, -0.8       ,
       -0.77777778, -0.75555556, -0.73333333, -0.71111111, -0.68888889,
       -0.66666667, -0.64444444, -0.62222222, -0.6       , -0.57777778,
       -0.55555556, -0.53333333, -0.51111111, -0.48888889, -0.46666667,
       -0.44444444, -0.42222222, -0.4       , -0.37777778, -0.35555556,
       -0.33333333, -0.31111111, -0.28888889, -0.26666667, -0.24444444,
       -0.22222222, -0.2       , -0.17777778, -0.15555556, -0.13333333,
       -0.11111111, -0.08888889, -0.06666667, -0.04444444, -0.02222222,
        0.        ,  0.02222222,  0.04444444,  0.06666667,  0.08888889,
        0.11111111,  0.13333333,  0.15555556,  0.17777778,  0.2       ,
        0.22222222,  0.24444444,  0.26666667,  0.28888889,  0.31111111,
        0.33333333,  0.35555556,  0.37777778,  0.4       ,  0.42222222,
        0.44444444,  0.46666667,  0.48888889,  0.51111111,  0.53

In [13]:
mean_fr = np.divide(hist[0], len(spk_times)*bin_size)
time = hist[1]
time_bins = (time[:-1] + time[1:])/2
# build gaussian kernel
if smoothing > 0:
    w = n_bins - 1 if n_bins % 2 == 0 else n_bins
    window = signal.gaussian(w, std=smoothing / bin_size)
    window /= np.sum(window)

psth = signal.convolve(mean_fr, window, mode='same', method='auto')

In [15]:
np.diff(time_bins)

array([0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222222,
       0.02222222, 0.02222222, 0.02222222, 0.02222222, 0.02222

In [49]:
def compute_psth(trials, trial_type, align_event, bin_size=0.025,
                 smoothing=0.025, x_lim=[-1, 1], as_dict=True):
    
    if trial_type == 'left':
        color = 'green'
    elif trial_type == 'right':
        color = 'blue'
    elif trial_type == 'all':
        color = 'black'
    elif trial_type == 'incorrect':
        color = 'red'
    else:
        raise NameError('Invalid type name')
        
    n_offset = 5 * int(np.ceil(smoothing / bin_size))  # get rid of boundary effects for smoothing
    n_bins_pre = int(np.ceil(np.negative(x_lim[0]) / bin_size)) + n_offset
    n_bins_post = int(np.ceil(x_lim[1] / bin_size)) + n_offset
    n_bins = n_bins_pre + n_bins_post

    spk_times = trials.fetch('trial_spike_times')
    hist = np.histogram(np.hstack(spk_times),
                        range=x_lim,
                        bins=n_bins)

    mean_fr = np.divide(hist[0], len(spk_times)*bin_size)
    time_bins = hist[1]
    # build gaussian kernel
    if smoothing > 0:
        w = n_bins - 1 if n_bins % 2 == 0 else n_bins
        window = signal.gaussian(w, std=smoothing / bin_size)
        window /= np.sum(window)

    psth = signal.convolve(mean_fr, window, mode='same', method='auto')

    data = go.Scatter(
        x=list(time_bins),
        y=list(psth),
        mode='lines',
        marker=dict(
            size=6,
            color=color),
        name='{} trials'.format(trial_type)
    )
    if as_dict:
        return data
    else:
        return list(time_bins), list(psth)

In [12]:
from ibl_pipeline import subject
keys = (ephys.Cluster & (subject.Subject & 'subject_nickname="CSHL_007"')).fetch('KEY')
key = keys[1]

In [50]:
cluster = ephys.Cluster & key
event = (ephys.Event & 'event="stim on"').fetch1('KEY')
trials_all = (behavior.TrialSet.Trial * ephys.TrialSpikes & cluster).proj(
    'trial_start_time', 'trial_stim_on_time', 'trial_response_time', 'trial_feedback_time',
    'trial_response_choice', 'trial_spike_times',
    trial_duration='trial_end_time-trial_start_time',
    trial_signed_contrast='trial_stim_contrast_right - trial_stim_contrast_left'
) & 'trial_duration < 5' & 'trial_response_choice!="No Go"' & event

trials_left = trials_all & 'trial_response_choice="CW"' & 'trial_signed_contrast < 0'
trials_right = trials_all & 'trial_response_choice="CCW"' & 'trial_signed_contrast > 0'
trials_incorrect = trials_all - trials_right.proj() - trials_left.proj()

align_event = (ephys.Event & key).fetch('event')

data = []
if len(trials_left):
    data.append(
        compute_psth(trials_left, 'left', align_event)
    )
if len(trials_right):
    data.append(
        compute_psth(trials_right, 'right', align_event)
    )
if len(trials_incorrect):
    data.append(
        compute_psth(trials_incorrect, 'incorrect', align_event)
    )

data.append(
    compute_psth(trials_all, 'all', align_event)
)

layout = go.Layout(
    width=580,
    height=370,
    margin=go.layout.Margin(
        l=50,
        r=30,
        b=40,
        t=80,
        pad=0
    ),
    title=dict(
        text='PSTH, aligned to {} time'.format(align_event),
        x=0.2,
        y=0.87
    ),
    xaxis=dict(
        title='Time (sec)',
        range=x_lim,
        showgrid=False
    ), 
    yaxis=dict(
        title='Firing rate (spks/sec)',
        showgrid=False
    ),
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [42]:
import json
f = open("psth_combined.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()

In [16]:
(ephys_plotting.PsthDataVarchar & key).fetch('psth_time')

array(['-0.91667,-0.75000,-0.58333,-0.41667,-0.25000,-0.08333,0.08333,0.25000,0.41667,0.58333,0.75000,0.91667',
       '-0.91667,-0.75000,-0.58333,-0.41667,-0.25000,-0.08333,0.08333,0.25000,0.41667,0.58333,0.75000,0.91667',
       '-0.91667,-0.75000,-0.58333,-0.41667,-0.25000,-0.08333,0.08333,0.25000,0.41667,0.58333,0.75000,0.91667'],
      dtype=object)

In [17]:
-0.91 +0.75

-0.16000000000000003